In [1]:
import pandas as pd

In [2]:
X_train = pd.read_pickle('../Data/model/X_train.pkl')
X_test = pd.read_pickle('../Data/model/X_test.pkl')

In [3]:
X_train.shape

(287, 500)

In [4]:
X_test.shape

(73, 500)

In [5]:
y_train = pd.read_pickle('../Data/model/y_train.pkl')
y_test = pd.read_pickle('../Data/model/y_test.pkl')

In [6]:
y_train.shape

(287,)

In [7]:
y_test.shape

(73,)

In [8]:
from sklearn.feature_selection import SelectFromModel

/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.utils._testing import ignore_warnings 
from sklearn.exceptions import ConvergenceWarning 

In [195]:
# logistic regression - https://stackoverflow.com/questions/38077190/how-to-increase-the-model-accuracy-of-logistic-regression-in-scikit-python
from sklearn.linear_model import LogisticRegression

lg_grid = {
    "penalty" : ['none', 'l1', 'l2', 'elasticnet'],
    # 'dual': ['True', 'False'],
    'tol': [1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3],
    "C" : [100, 10, 1.0, 0.1, 0.01],
    # 'fit_intercept': ['True', 'False'],
    "solver" : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    "max_iter": [50, 100, 500, 1000, 5000, 10000, 20000]
}

with ignore_warnings(): 
    lg_search = RandomizedSearchCV(
        estimator = LogisticRegression(class_weight='balanced', random_state = 1), 
        param_distributions = lg_grid, 
        random_state = 1, 
        n_iter = 60,
        return_train_score = True,
        error_score = 0
    )

    lg_model = lg_search.fit(X_train, y_train)
    print(lg_search.best_params_)
    print(lg_search.best_score_)

{'tol': 0.001, 'solver': 'saga', 'penalty': 'none', 'max_iter': 100, 'C': 1.0}
0.6136116152450091


In [11]:
lg_pred = lg_model.predict(X_test)

NameError: name 'lg_model' is not defined

In [190]:
X_test.columns.equals(X_train.columns)

True

In [ ]:
# lg_model = SelectFromModel(lg_result.best_estimator_, prefit=True)
# lg_model.transform(X_train)

In [ ]:
print(classification_report(lg_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.75      0.56      0.64        48
         1.0       0.43      0.64      0.52        25

    accuracy                           0.59        73
   macro avg       0.59      0.60      0.58        73
weighted avg       0.64      0.59      0.60        73



In [199]:
# linear discriminant analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_grid = {
    "solver": ['svd', 'lsqr', 'eigen'],
    "shrinkage": ['auto', 'float']
}

with ignore_warnings():
    lda_search = RandomizedSearchCV(
        estimator = LogisticRegression(random_state = 1), 
        param_distributions = lg_grid, 
        return_train_score = True, 
        random_state = 1, 
        n_iter = 60
    )

    lda_model = lda_search.fit(X_train, y_train)
    print(lda_model.best_params_)
    print(lda_model.best_score_)

In [ ]:
lda_pred = lda_model.predict(X_test)

In [ ]:
print(classification_report(lda_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.69      0.52      0.60        48
         1.0       0.38      0.56      0.45        25

    accuracy                           0.53        73
   macro avg       0.54      0.54      0.52        73
weighted avg       0.59      0.53      0.55        73



In [2]:
# k-nearest neighbors
from sklearn.neighbors import KNeighborsClassifier
# knn_model = KNeighborsClassifier(n_neighbors=15)

knn_grid = {
    
}

with ignore_warnings():    
    knn_model.fit(X_train, y_train)
    knn_pred = knn_model.predict(X_test)

NameError: name 'ignore_warnings' is not defined

In [ ]:
print(classification_report(knn_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.42      0.47      0.44        32
         1.0       0.54      0.49      0.51        41

    accuracy                           0.48        73
   macro avg       0.48      0.48      0.48        73
weighted avg       0.49      0.48      0.48        73



In [ ]:
# naive-bayes
from sklearn.naive_bayes import GaussianNB

nb_grid = {
    "var_smoothing": [1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

nb_search = RandomizedSearchCV(estimator=GaussianNB(), param_distributions=nb_grid, return_train_score=True, random_state=1, n_jobs=-1, n_iter=50)
nb_result = nb_search.fit(X_train, y_train)
nb_pred = nb_result.predict(X_test)

/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=50. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
print(classification_report(nb_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.83      0.49      0.62        61
         1.0       0.16      0.50      0.24        12

    accuracy                           0.49        73
   macro avg       0.50      0.50      0.43        73
weighted avg       0.72      0.49      0.56        73



In [ ]:
# decision tree
from sklearn.tree import DecisionTreeClassifier

grid = {
    "criterion" : ['gini', 'entropy', 'log_loss'],
    "splitter": ['best', 'random'],
    "max_depth" : [5, 10, 25, 50, 100, 500],
    "max_features" : ['auto', 'sqrt', 'log2'] 
}

dt_search = RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=1), param_distributions=grid, return_train_score=True, random_state=1, n_jobs=-1, n_iter=25)
dt_result = dt_search.fit(X_train, y_train)
dt_pred = dt_result.predict(X_test)

/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, exp

In [ ]:
print(classification_report(dt_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.69      0.60      0.64        42
         1.0       0.54      0.65      0.59        31

    accuracy                           0.62        73
   macro avg       0.62      0.62      0.61        73
weighted avg       0.63      0.62      0.62        73



In [ ]:
from sklearn.svm import SVC

grid = {
    "kernel" : ['polynomial', 'gaussian', 'linear'],
    "C" : [100, 10, 1.0, 0.1, 0.01],
    "gamma": ['scale', 'auto'],
    "degree": [2, 3, 4, 5, 6],
    "shrinking": [True, False]
}

svc_search = RandomizedSearchCV(estimator=SVC(random_state=1), param_distributions=grid, return_train_score=True, random_state=1, n_jobs=-1, n_iter=25)
svc_result = svc_search.fit(X_train, y_train)
svc_pred = svc_result.predict(X_test)

/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
75 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/svm/_base.py", line 251, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/Users/isobarbaric/miniforge3/lib/python3.9/site-packages/sklearn/svm/_base.py", line 333, in _de

In [ ]:
print(classification_report(svc_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.69      0.52      0.60        48
         1.0       0.38      0.56      0.45        25

    accuracy                           0.53        73
   macro avg       0.54      0.54      0.52        73
weighted avg       0.59      0.53      0.55        73



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

<style>
  .center {
    display: block;
    margin-left: auto;
    margin-right: auto;
  }
</style>
<img src="https://miro.medium.com/max/1838/1*fxiTNIgOyvAombPJx5KGeA.png"  height="300" class='center'/>

In [ ]:
columns = index = ['scientific', 'conspiracy']

In [ ]:
def get_classification_report_html(model_predictions):
    report = classification_report(y_test, model_predictions, output_dict=True)
    df = pd.DataFrame(report).transpose()
    return df.to_html()

def get_confusion_matrix_html(model_predictions):
    return pd.DataFrame(confusion_matrix(y_test, model_predictions), index=index, columns=columns).to_html()

prediction content starts here

In [ ]:
import json
from bagOfWords import BagOfWords

def build_predict_dataframe(test_content):
    with open('../Data/model/relevant_words.json') as f:
        relevant_words = json.loads(f.read())

    current_test = BagOfWords(test_content, None)

    cols = {}
    for word in relevant_words:
        cols[word] = [current_test.freqChart[word] if word in current_test.freqChart else 0]
    
    data_set = pd.DataFrame(data = cols)

    return data_set

In [ ]:
s = "Jacob Puliyel, MD, a pediatrician in India for more than 40 years, brought suit in the Supreme Court of India against the Union of India and COVID-19 vaccine manufacturers in a legal challenge to the country\u2019s COVID vaccine program.1 Dr. Puliyel, who has served as Director of Research and Projects at Holy Family Hospital in Delhi and is a former member of the National Technical Advisory Group (NTAG) on immunizations in India, sued the government and COVID vaccine manufacturers seeking release of information related to the COVID vaccine approval process, as well as arguing for a policy change that allows unvaccinated persons to enter public spaces and access resources.\n\nSpecifically, Dr. Puliyel asked the Supreme Court of India for the release of each phase of clinical trial data for the COVID vaccines administered in India; disclosure of minutes from the meeting of the Subject Expert Committee and the NTGAI with regard to vaccines; release of information surrounding the approval or rejection of emergency use applications of vaccines by the Drugs Controller General of India (DCGI); and disclosure of post vaccination data related to COVID. He also asked the court to declare that all COVID vaccine mandates are unconstitutional and a violation of h2citizen\u2019s rights.\n\nDr. Puliyel: COVID Vaccines Do Not Prevent Infection or Transmission\n\nIn support of his request, Dr. Puliyel submitted affidavits to the court stating that natural immunity from SARS-CoV-2 is superior to vaccine acquired immunity and that vaccines do not prevent infection or transmission of COVID. News articles were submitted to show that two-thirds of the population in India had already been infected with SARS-CoV-2 and have antibodies to the virus, and that the vaccinated were susceptible to breakthrough infections. Dr. Puliyel also argued that vaccine mandates and restrictions on the unvaccinated are meaningless because the virus is transmitted by the vaccinated and unvaccinated alike.\n\nThe government maintained that the COVID vaccines administered in India, Covaxin and Covishield, are needed to combat the global pandemic threat. They also maintained that trial data cannot be released to the public due to privacy concerns; that information regarding adverse events has already been released; that vaccination remains voluntary for all citizens, and that restrictions on the unvaccinated were necessary to protect the public health.\n\nRespondent COVID vaccine manufacturers argued that they followed strict guidelines for government approval of their COVID vaccines.\n\nThe Union of India strongly opposed this lawsuit, claiming that the publicity could increase vaccine hesitancy and endanger the public health, and that public health matters should not be interfered with by the court. However, because Dr. Puliyel brought this lawsuit in the interest of the public and because he is considered a subject matter expert, the Court determined the case would be heard for the welfare of the public.\n\nThe Supreme Court of India Looked to High Courts in Other Countries\n\nThe government made their argument relying on prominent cases in India, as well as U.S. cases, such as Jacobson v. Commonwealth of Massachusetts, Zucht v. King; Biden vs. Missouri, and Roman Catholic Diocese v. Cuomo. The court also looked to New South Wales for guidance, finding that the Supreme Court in Assam v. Hazzard held that it was not the court\u2019s role to determine the effectiveness of the government\u2019s regulations or whether the regulations were the best response to the risks COVID posed to the public, instead the New South Whales Supreme Court emphasized that their only role was to determine the legal validity of the Minister\u2019s orders.\n\nReviewing India\u2019s case history, the Court summarized its role in reviewing policy decisions:\n\nto check whether it violates the fundamental rights of the citizens or is opposed to the provisions of the Constitution, or opposed to any statutory provision or manifestly arbitrary\u201d but not to, \u201csubstitute their own views as to what is wise, safe, prudent or proper. 2 In Distribution of Essential Supplies & Services during Pandemic, the Court determined that it would not, \u201csecond guess\u201d the policy experts, but would \u201cexercise jurisdiction to determine if the chosen policy measure conforms to the standards of reasonableness, militates against manifest arbitrariness and protects the right to life of all persons.\u201d3\n\nCourt Found Forced Vaccination Violates the Constitution of India\n\nIn considering the global pandemic, the court said that it could not find fault with the government\u2019s vaccination drive, citing the highly transmissible Delta and Omicron variants of SARS-CoV-2; the approximate loss of 5,23,843 lives from the virus in India; and the recommendations of the World Health Organization (WHO), TAG-CO-VAC and SAGE that vaccination would provide strong protection against serious illness, hospitalization and death.\n\nHowever, regarding the issue of the right of individual privacy verses the government restricting individual rights in the name of public health, the Supreme Court of India found that forced vaccination is a violation of Article 21 of the Constitution of India. The Court previously found in Article 21 a right to personal autonomy, which extends to the right to refuse medical treatment.\n\nIn balancing the state\u2019s interest in public health and an individual\u2019s right to personal autonomy, the court found that Article 21 of the Indian Constitution grants individuals the right to determine how they should live their own life, which includes the right to refuse medical treatment such as vaccination. However, when there is a likelihood that an individual could spread the virus to other people, the state\u2019s interest in protecting the public allows the government to impose certain restrictions on individual rights, as long as such restrictions are reasonable, in proportion to the objective sought, and reviewable by the courts. Accordingly, the Court found that the government\u2019s restrictions are a valid exercise of power, despite evidence that an unvaccinated person is no more likely to spread the Delta or Omicron variants of the SARS-CoV-2 virus than a vaccinated person.\n\nThe Court: COVID Vaccination Policy in India Not Unreasonable\n\nIn support of his case, Dr. Puliyel presented scientific journals showing that patients who recovered from COVID still had antibodies 11 months after infection. He also pointed out that countries with a higher percent of the population vaccinated have a higher rate of COVID per one million people. He said that a vaccine surveillance report in the United Kingdom showed that, in week 40, there was a negative efficacy of the vaccine in those over the age of 30.\n\nThe court confirmed that its only role is to determine whether the Union of India took into consideration scientific research when establishing its policy advocating COVID vaccination for the entire country. The court acknowledged that large portions of the public cannot be considered healthy because they suffer from chronic disease and lack of nutrition, especially the elderly population, and that the Union of India\u2019s vaccination policy was rightly centered around large segments of more vulnerable people, rather than a minority who are healthy as the subjects in the scientific articles that Dr. Puliyel presented to support the efficacy of natural immunity. Accordingly, the court found that the COVID vaccination policy of India was made in the interest of public health and is not unreasonable.\n\nThe Court: Vaccinated Population a Source of Virus Transmission\n\nConsidering the fact that the vaccinated and the unvaccinated pose nearly the same risk of transmitting the virus, the court looked at whether the Union of India\u2019s vaccination policy of restricting unvaccinated persons from accessing public resources amounted to discrimination. Dr. Puliyel presented a letter published in the Lancet finding that peak viral load of COVID did not differ either by vaccination status or by variant; that the secondary infection rate of vaccinated and unvaccinated households only differed by two percent, and that four of the five most vaccinated countries were categorized by the U.S. Centers for Disease Control and Prevention (CDC) as high transmission countries. In addition, he submitted a study conducted in Massachusetts in July 2021 demonstrating that, after attending public gatherings, 346 out of 469 COVID cases (74 percent) were among vaccinated persons.\n\nPresented with this data, the court concluded:\n\nIt appears to be grossly negligent to ignore the vaccinated population as a possible and relevant source of transmission when deciding about public health control measures.4 The court admitted that vaccine mandates may have been found to be constitutional prior to the emergence of SARS-CoV-2 variants that do not appear to respond to COVID vaccines. However, now that vaccinated persons present nearly the same risk of transmission as unvaccinated persons, the court said that restrictions placed on the unvaccinated are not in proportion to the risk they pose to public health.4\n\nThe court wrote:\n\nWhile there is abundant data to show that getting vaccinated continues to be the dominant expert advice even in the face of new variants, no submission nor any data has been put forth to justify restrictions only on unvaccinated individuals when emerging scientific evidence appears to indicate that the risk of transmission of the virus from unvaccinated individuals is almost on par with that from vaccinated persons. To put it differently, neither the Union of India nor the State Governments have produced any material before this court to justify the discriminatory treatment of unvaccinated individuals in public places by imposition of vaccine mandates\u201d5\n\nThe court suggested that all authorities, public, private and educational, review their current orders that restrict an unvaccinated person\u2019s access to public services and resources.\n\nThe Court: Government Followed Approval Process for COVID Vaccines\n\nDr. Puliyel asserted that the government rushed the approval of COVID vaccines in, \u201ca hurried and opaque manner.\u201d6 He said that the trial data, regulatory approvals, and minutes from meetings approving the vaccine have not been publicly disclosed, which demonstrates a lack of transparency in the approval process, and that, while the complete data sets may not be available yet, releasing preliminary COVID vaccine adverse event data would allow individuals to make an informed vaccination decision.\n\nThe court disagreed, finding that the government\u2019s approval for emergency use of the COVID-19 vaccines was properly made in accordance with the statutory regime in place. In India, all new drug trials must follow a certain protocol with stringent statutory requirements as set forth in the Good Clinical Practices guidelines and be approved by the Ethics Committee and the Central Licensing Authority. The Ethics Committee, which is comprised of medical, scientific and non-medical and nonscientific members, is an independent review board that is tasked with providing an objective and independent review of the clinical trials.\n\nThe Union of India claimed that these strict clinical trial guidelines were followed for COVID vaccine clinical trials and that the main findings of trials have been published in a peer reviewed journal, in accordance with WHO requirements.\n\nThe Court: Ongoing COVID Vaccine Trial Data Must Be Publicly Available\n\nThe court agreed with the government and declined to mandate the disclosure of segregated clinical trial data, citing privacy reasons, and also declined to declare that COVID vaccine approvals were granted without proper review of clinical trial data. However, the court ordered that relevant data, which must be published under statutory regimes and WHO requirements for ongoing clinical trials, along with data from COVID vaccine post-marketing trials, be made available to the public without delay.\n\nDr. Puliyel advised the court that, because clinical trials are made up of a small subset of the population, not all adverse events may be seen during trials and, instead, may only be noticed once the vaccine is available to the public at large. He pointed out that, in 2018, the WHO changed the way it classified adverse events after vaccination to only consider adverse events as vaccine related when that same adverse reaction also occurred during previous smaller, limited clinical trials.\n\nIn other words:\n\nReactions observed during post-marketing surveillance are not considered as \u2018consistent with causal association with vaccine\u2019, if a significant increase in such reactions during Phase III trials had not been recorded.7\n\nIn the case of the COVID vaccine trials, phase III trials were cut short and then control subjects were offered the vaccine, which had the effect of diluting the control group and ultimately making it impossible to evaluate potential differences in long term health outcomes between the vaccinated and control groups. At the same time, it makes it more possible for doctors to dismiss adverse events that occur post COVID vaccination, as adverse events are less likely to be considered vaccine related events. Therefore, Dr. Puliyel asked that all post marketing adverse event data be made available to the public.\n\nHowever, the court declined to act, finding that the Phase III trials were cut short on the advice of the SEC, and stating that the government has protocols in place to consider and investigate adverse reactions to COVID vaccine, which only occur during the post-marketing phase.\n\nDr. Puliyel: India\u2019s Adverse Event Reporting System Should Be More Like VAERS\n\nDr. Puliyel contended that the vaccine adverse events reporting system in India is not transparent and complete data as on serious adverse events should be published like in the United States though the Vaccine Adverse Event Reporting System (VAERS). The court pointed out that the National AEFI Surveillance Secretariat, which has been in place for a decade, has a protocol in place for identifying and monitoring adverse events to vaccination.\n\nHowever, Dr. Puliyel argued that the vaccine adverse event reporting system in India needs to be changed to allow individuals, as well as private doctors, to report adverse events like is done in the U.S. with VAERS. The court agreed and directed the Union of India to facilitate the reporting of vaccine adverse events by individuals, as well as private doctors, on a public on-line platform. The court said:\n\nInformation relating to adverse effects following immunisation is crucial for the purpose of understanding the safety of the vaccines that are being administered, apart from being instrumental in further scientific studies around the pandemic. There is an imminent need for collection of requisite data of adverse events and wider participation of people in reporting the adverse events is necessary for the purpose of gathering correct information.8\n\nThe Court: Pediatric COVID-19 Vaccination Program Will Stand\n\nDr. Puliyel presented evidence that children are not at high risk of severe illness from COVID and argued that the risk from COVID vaccination outweighs the risk of getting seriously ill from the virus, pointing out that a large number of children in India have already recovered from a SARS-CoV-2 infection.\n\nThe court sided with the government, stating that it would not second-guess expert opinion as to the safety of pediatric COVID vaccinations. However, the court ordered the Union of India to immediately release the key findings and clinical trial results of Corbevax, the COVID vaccine that does not use messenger RNA (mRNA) technology, which is being given to children, if that data has not already been released.\n\nThe Supreme Court of India acknowledged that citizens have a liberty interest that prohibits forced vaccination; that restrictions placed on unvaccinated persons should be reviewed in light of the fact that the COVID vaccine does not prevent infection and transmission; that ongoing COVID vaccine clinical trial data should be publicly released, and that the government\u2019s vaccine adverse event reporting system should be improved. However, the court did not fully recognize and adequately respond to all of Dr. Puliyel\u2019s legitimate complaints about the government\u2019s COVID vaccination program.\n\nIf you would like to receive an e-mail notice of the most recent articles published in The Vaccine Reaction each week, click here.\n\nClick here to view References:"

In [ ]:
p_mat = build_predict_dataframe(s)

In [ ]:
p_mat.loc[0].unique()

array([ 0,  1,  2, 22,  9,  8,  4,  3,  7, 12,  6,  5])

In [ ]:
len(p_mat.columns)

50

In [ ]:
lg_result.predict(p_mat)

array([1.])

In [ ]:
nb_result.predict_proba(p_mat)

array([[9.99999699e-01, 3.00619551e-07]])

In [ ]:
for model in ['lg', 'lda', 'knn', '']:
    print(model)

lg
lda
knn



results

<table>
    <thead>
        <tr>
            <th></th>
            <th rowspan="2">Confusion Matrix</th>
            <th rowspan="2">Classification Report</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>Logistic Regression</th>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>positive</th>
                        <th>negative</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>positive</th>
                        <td>34</td>
                        <td>18</td>
                        </tr>
                        <tr>
                        <th>negative</th>
                        <td>5</td>
                        <td>47</td>
                        </tr>
                    </tbody>
                </table>
            </td>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>precision</th>
                        <th>recall</th>
                        <th>f1-score</th>
                        <th>support</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>0.0</th>
                        <td>0.871795</td>
                        <td>0.653846</td>
                        <td>0.747253</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>1.0</th>
                        <td>0.723077</td>
                        <td>0.903846</td>
                        <td>0.803419</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>accuracy</th>
                        <td>0.778846</td>
                        <td>0.778846</td>
                        <td>0.778846</td>
                        <td>0.778846</td>
                        </tr>
                        <tr>
                        <th>macro avg</th>
                        <td>0.797436</td>
                        <td>0.778846</td>
                        <td>0.775336</td>
                        <td>104.000000</td>
                        </tr>
                        <tr>
                        <th>weighted avg</th>
                        <td>0.797436</td>
                        <td>0.778846</td>
                        <td>0.775336</td>
                        <td>104.000000</td>
                        </tr>
                    </tbody>
                </table>            
            </td>
        </tr>
        <tr>
            <th>Linear Discriminant Analysis</th>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>positive</th>
                        <th>negative</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>positive</th>
                        <td>37</td>
                        <td>15</td>
                        </tr>
                        <tr>
                        <th>negative</th>
                        <td>7</td>
                        <td>45</td>
                        </tr>
                    </tbody>
                </table>
            </td>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>precision</th>
                        <th>recall</th>
                        <th>f1-score</th>
                        <th>support</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>0.0</th>
                        <td>0.725000</td>
                        <td>0.557692</td>
                        <td>0.630435</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>1.0</th>
                        <td>0.640625</td>
                        <td>0.788462</td>
                        <td>0.706897</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>accuracy</th>
                        <td>0.673077</td>
                        <td>0.673077</td>
                        <td>0.673077</td>
                        <td>0.673077</td>
                        </tr>
                        <tr>
                        <th>macro avg</th>
                        <td>0.682813</td>
                        <td>0.673077</td>
                        <td>0.668666</td>
                        <td>104.000000</td>
                        </tr>
                        <tr>
                        <th>weighted avg</th>
                        <td>0.682812</td>
                        <td>0.673077</td>
                        <td>0.668666</td>
                        <td>104.000000</td>
                        </tr>
                    </tbody>
                </table>
            </td>
        </tr>
        <tr>
            <th>KNN</th>
            <td>
                <table border="1" class="dataframe">
                <thead>
                    <tr style="text-align: right;">
                    <th></th>
                    <th>positive</th>
                    <th>negative</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                    <th>positive</th>
                    <td>23</td>
                    <td>29</td>
                    </tr>
                    <tr>
                    <th>negative</th>
                    <td>20</td>
                    <td>32</td>
                    </tr>
                </tbody>
                </table>
            </td>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>precision</th>
                        <th>recall</th>
                        <th>f1-score</th>
                        <th>support</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>0.0</th>
                        <td>0.534884</td>
                        <td>0.442308</td>
                        <td>0.484211</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>1.0</th>
                        <td>0.524590</td>
                        <td>0.615385</td>
                        <td>0.566372</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>accuracy</th>
                        <td>0.528846</td>
                        <td>0.528846</td>
                        <td>0.528846</td>
                        <td>0.528846</td>
                        </tr>
                        <tr>
                        <th>macro avg</th>
                        <td>0.529737</td>
                        <td>0.528846</td>
                        <td>0.525291</td>
                        <td>104.000000</td>
                        </tr>
                        <tr>
                        <th>weighted avg</th>
                        <td>0.529737</td>
                        <td>0.528846</td>
                        <td>0.525291</td>
                        <td>104.000000</td>
                        </tr>
                    </tbody>
                </table>
            </td>
        </tr>
        <tr>
            <th>GaussianNB</th>
            <td>
                <table border="1" class="dataframe">
                <thead>
                    <tr style="text-align: right;">
                    <th></th>
                    <th>positive</th>
                    <th>negative</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                    <th>positive</th>
                    <td>37</td>
                    <td>15</td>
                    </tr>
                    <tr>
                    <th>negative</th>
                    <td>7</td>
                    <td>45</td>
                    </tr>
                </tbody>
                </table>
            </td>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>precision</th>
                        <th>recall</th>
                        <th>f1-score</th>
                        <th>support</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>0.0</th>
                        <td>0.840909</td>
                        <td>0.711538</td>
                        <td>0.770833</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>1.0</th>
                        <td>0.750000</td>
                        <td>0.865385</td>
                        <td>0.803571</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>accuracy</th>
                        <td>0.788462</td>
                        <td>0.788462</td>
                        <td>0.788462</td>
                        <td>0.788462</td>
                        </tr>
                        <tr>
                        <th>macro avg</th>
                        <td>0.795455</td>
                        <td>0.788462</td>
                        <td>0.787202</td>
                        <td>104.000000</td>
                        </tr>
                        <tr>
                        <th>weighted avg</th>
                        <td>0.795455</td>
                        <td>0.788462</td>
                        <td>0.787202</td>
                        <td>104.000000</td>
                        </tr>
                    </tbody>
                </table>
            </td>
        </tr>
        <tr>
            <th>Decision Tree Classifier</th>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>positive</th>
                        <th>negative</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>positive</th>
                        <td>33</td>
                        <td>19</td>
                        </tr>
                        <tr>
                        <th>negative</th>
                        <td>17</td>
                        <td>35</td>
                        </tr>
                    </tbody>
                </table>
            </td>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>precision</th>
                        <th>recall</th>
                        <th>f1-score</th>
                        <th>support</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>0.0</th>
                        <td>0.660000</td>
                        <td>0.634615</td>
                        <td>0.647059</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>1.0</th>
                        <td>0.648148</td>
                        <td>0.673077</td>
                        <td>0.660377</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>accuracy</th>
                        <td>0.653846</td>
                        <td>0.653846</td>
                        <td>0.653846</td>
                        <td>0.653846</td>
                        </tr>
                        <tr>
                        <th>macro avg</th>
                        <td>0.654074</td>
                        <td>0.653846</td>
                        <td>0.653718</td>
                        <td>104.000000</td>
                        </tr>
                        <tr>
                        <th>weighted avg</th>
                        <td>0.654074</td>
                        <td>0.653846</td>
                        <td>0.653718</td>
                        <td>104.000000</td>
                        </tr>
                    </tbody>
                </table>
            </td>
        </tr>     
        <tr>
            <th>Support Vector Machine</th>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>positive</th>
                        <th>negative</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>positive</th>
                        <td>32</td>
                        <td>20</td>
                        </tr>
                        <tr>
                        <th>negative</th>
                        <td>10</td>
                        <td>42</td>
                        </tr>
                    </tbody>
                </table>            
            </td>
            <td>
                <table border="1" class="dataframe">
                    <thead>
                        <tr style="text-align: right;">
                        <th></th>
                        <th>precision</th>
                        <th>recall</th>
                        <th>f1-score</th>
                        <th>support</th>
                        </tr>
                    </thead>
                    <tbody>
                        <tr>
                        <th>0.0</th>
                        <td>0.761905</td>
                        <td>0.615385</td>
                        <td>0.680851</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>1.0</th>
                        <td>0.677419</td>
                        <td>0.807692</td>
                        <td>0.736842</td>
                        <td>52.000000</td>
                        </tr>
                        <tr>
                        <th>accuracy</th>
                        <td>0.711538</td>
                        <td>0.711538</td>
                        <td>0.711538</td>
                        <td>0.711538</td>
                        </tr>
                        <tr>
                        <th>macro avg</th>
                        <td>0.719662</td>
                        <td>0.711538</td>
                        <td>0.708847</td>
                        <td>104.000000</td>
                        </tr>
                        <tr>
                        <th>weighted avg</th>
                        <td>0.719662</td>
                        <td>0.711538</td>
                        <td>0.708847</td>
                        <td>104.000000</td>
                        </tr>
                    </tbody>
                </table>
            </td>
        </tr>
    </tbody>
</table>